### Midlevel Moisture Timeseries Plot

Emily Luschen \
Emily.W.Luschen-1@ou.edu \
28 Aug 2025

In [1]:
# import packages 
import numpy as np # libary for working with arrays
from grl_mask_tc_track_3d import mask_tc_track_3d # function to mask out the TC track
import grl_functions as fun # custom functions
import xarray as xr
import pickle

Edited: 1 May 2025
Goal: Three panel plot
* Midlevel relative humidity

To Do:
1. Read in qvap, T, lat, lon
2. Calculate RH
3. Mask RH
4. Average over RH
5. Pickle RH

In [2]:
storm = 'maria' # or 'maria'
ntime = 49 

if storm == 'haiyan':
    wrfout = wrf_path # for lat and lon
    ntest = 4
    tests = ['ncrf36h', 'STRATANVIL_OFF', 'STRATANVIL_ON', 'STRAT_OFF']
    nmem = 8
elif storm == 'maria': # maria
    wrfout = wrf_path_maria
    ntest = 1
    tests = ['ncrf48h'] 
    nmem = 10



# data parameters
p_lev = 16 # only one level available
t0 = 0 # start time index
t1 = 49 # end time index
ntime = 49 # senstivity test time length
membs = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

# TC tracking parameters
ptrack='600' # tracking pressure level
var_track = 'rvor' # variable
rmax = 3 # radius (deg) limit for masking around TC center

In [3]:
# 1. Load in lat and lon

varname='XLAT'
lat = fun.var_wrfread(wrfout,varname) # latitude
varname='XLONG'
lon = fun.var_wrfread(wrfout,varname) # latitude

nlat, nlon = np.shape(lat)

### CTL Loop

In [ ]:
# ctl
rh_mean_ctl = np.zeros((nmem, 97)) 

for m in range(nmem): # member loop
    # 1. Load in data: qvap, temp, pres
    qvapfile = qvap_path # maria
    Tfile = temp_path # maria
    df = xr.open_mfdataset((qvapfile,Tfile))
    qvap = df.QVAPOR.data[:,p_lev,:,:] # time, level, lat, lon
    temp = df.T.data[:,p_lev,:,:]
    pres = df.pres.data[p_lev]
    df.close()

    # 2. Calculate RH
    rh = fun.relh(qvap, pres*100, temp-273.15) # change pressure to Pa and temp to C

    # 3. Mask RH
    track_file = tracking_path # maria
    rh_mask = mask_tc_track_3d(track_file, rmax, rh, lon, lat, t0=0, t1=97)

    # 4. Average over lat and lon and level
    rh_mean_ctl[m,:] = np.ma.mean(rh_mask, axis=(1,2))

    print('Member '+membs[m]+' complete')

# estimated completion time: 5m 3s

In [5]:
# intialize arrays
rh_mean = np.zeros((ntest,nmem,ntime))*np.nan

for itest in range(ntest): # test loop
    print('Now running test: '+tests[itest])
    for m in range(nmem): # member loop
        # 1. Load in data: qvap, temp, pres
        qvapfile = qvap_path
        Tfile = temp_path
        df = xr.open_mfdataset((qvapfile,Tfile))
        qvap = df.QVAPOR.data[:,p_lev,:,:] # time, level, lat, lon
        temp = df.T.data[:,p_lev,:,:]
        pres = df.pres.data[p_lev]
        df.close()

        # 2. Calculate RH
        rh = fun.relh(qvap, pres*100, temp-273.15)

        # 3. Mask out TC track
        track_file = tracking_path # use ncrf track
        rh_mask = mask_tc_track_3d(track_file, rmax, rh, lon, lat, t0=0, t1=49) # ncrf track

        # 4. Average over lat and lon and level
        rh_mean[itest,m,:] = np.ma.mean(rh_mask, axis=(1,2))

        print('Member '+membs[m]+' complete')
    print('Test '+tests[itest]+' complete')

    # estimated completion time: 31m 48s

Now running test: ncrf48h
Member 01 complete
Member 02 complete
Member 03 complete
Member 04 complete
Member 05 complete
Member 06 complete
Member 07 complete
Member 08 complete
Member 09 complete
Member 10 complete
Test ncrf48h complete


In [7]:
# Write out to pickle file
pickle_file = pickle_path
with open(pickle_file, 'wb') as file:
    pickle.dump([rh_mean_ctl, rh_mean], file)
    # rh_mean_ctl: (nmem, time)
    # rh_mean: (ntest, nmem, time)